In [18]:
import time
import pandas as pd
import os
import glob
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver import Chrome
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

class HardwarezoneCrawler:
    def __init__(self) -> None:
        options = webdriver.ChromeOptions() 
        options.add_argument('log-level=3')
        options.add_argument("start-maximized")
        options.add_argument("--disable-gpu")
        options.add_argument("enable-automation")
        options.add_argument("--headless")
        options.add_argument("--no-sandbox")
        options.add_argument("--disable-dev-shm-usage")
        options.add_argument("--disable-browser-side-navigation")
        options.add_experimental_option("excludeSwitches", ["enable-automation"])
        options.add_experimental_option('useAutomationExtension', False)
        options.add_experimental_option('excludeSwitches', ['enable-logging'])
        self.driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)
    
    def get_options():
        options = webdriver.ChromeOptions() 
        options.add_argument('log-level=3')
        options.add_argument("start-maximized")
        options.add_argument("--disable-gpu")
        options.add_argument("enable-automation")
        options.add_argument("--headless")
        options.add_argument("--no-sandbox")
        options.add_argument("--disable-dev-shm-usage")
        options.add_argument("--disable-browser-side-navigation")
        options.add_experimental_option("excludeSwitches", ["enable-automation"])
        options.add_experimental_option('useAutomationExtension', False)
        options.add_experimental_option('excludeSwitches', ['enable-logging'])
        return options
        
    def get_url(self, aspect_item: str):
        self.driver.quit()
        options = webdriver.ChromeOptions() 
        options.add_argument('log-level=3')
        options.add_argument("start-maximized")
        options.add_argument("--disable-gpu")
        options.add_argument("enable-automation")
        options.add_argument("--headless")
        options.add_argument("--no-sandbox")
        options.add_argument("--disable-dev-shm-usage")
        options.add_argument("--disable-browser-side-navigation")
        options.add_experimental_option("excludeSwitches", ["enable-automation"])
        options.add_experimental_option('useAutomationExtension', False)
        options.add_experimental_option('excludeSwitches', ['enable-logging'])
        self.driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)
        self.driver.get("https://www.hardwarezone.com.sg/search/forum")

        # Wait for the search input field to be clickable
        search_navi = WebDriverWait(self.driver, 11).until(
            EC.element_to_be_clickable((By.XPATH, "//form/input[@class='text']"))
        )
        search_navi.send_keys(aspect_item)
        time.sleep(10)

        submit_btn = self.driver.find_element("xpath","//form/input[@type='submit']")
        webdriver.ActionChains(self.driver).move_to_element(submit_btn).click(submit_btn).perform()
        time.sleep(10)
        forum_btn = self.driver.find_element("xpath","//ul[@class='tabs-menu']/li[6]")
        webdriver.ActionChains(self.driver).move_to_element(forum_btn).click(forum_btn).perform()
        time.sleep(10)

        self.driver.execute_script ("window.scrollTo(0,document.body.scrollHeight);")

        # The first page of forums
        threads_list = []
        links_list = self.driver.find_elements("xpath","//a[@class='gs-title']")
        for link in links_list:
            if link.get_attribute('href'):
                threads_list.append(link.get_attribute('href'))
                
        # The rest pages of forums
        for i in range(2, 11):
            try:
                page_ele = self.driver.find_element("xpath", "//div[@aria-label='Page " + str(i) + "']")
                webdriver.ActionChains(self.driver).move_to_element(page_ele).click(page_ele).perform()
                time.sleep(10)
            except NoSuchElementException:
                print("Page element not found for page", i)
                a=input()
            
            links_list = self.driver.find_elements("xpath","//a[@class='gs-title']")
            for link in links_list:
                if link.get_attribute('href'):
                    threads_list.append(link.get_attribute('href'))

        # Navigate to Google
        try:
            google_navi_ele = self.driver.find_element("xpath","//div[@class='gcsc-more-maybe-branding-root']/a")
            google_search_url = google_navi_ele.get_attribute("href")
            self.driver.get(google_search_url)
        except NoSuchElementException:
            print("Page element not found for page")

        # The first page of google search results
        links_list = self.driver.find_elements("xpath","//div[@class='yuRUbf']/a")
        for link in links_list:
            if link.get_attribute("href"):
                threads_list.append(link.get_attribute("href"))

        self.driver.execute_script ("window.scrollTo(0,document.body.scrollHeight);")
        # The rest pages of google search results
        for i in range(2, 16):
            try:
                page_ele = self.driver.find_element("xpath","//a[@aria-label='Page " + str(i) + "']")
                webdriver.ActionChains(self.driver).move_to_element(page_ele).click(page_ele).perform()
                time.sleep(10)
                
                links_list = self.driver.find_elements("xpath","//div[@class='yuRUbf']/a")
                for link in links_list:
                    if link.get_attribute('href'):
                        threads_list.append(link.get_attribute('href'))
            except:
                print("We can not find the pages that required, end the loop")
                    
        threads_list = list(set(threads_list))
        aspects_list = [aspect_item for i in range(len(threads_list))]
        data_df = pd.DataFrame(list(zip(threads_list, aspects_list)), columns=['url', 'aspect'])
        data_df.to_csv("E:/Critical Inquiry/Code/Data Scraping/data/hwz_data_" + aspect_item + ".csv", index=False)
    
    def getReviews_content(self, csv_path: str, aspect_term: str):
        # Initialize WebDriver instance outside the loop
        options = webdriver.ChromeOptions() 
        options.headless = True
        options.add_argument('log-level=3')
        options.add_argument("start-maximized")
        options.add_argument("--disable-gpu")
        options.add_argument("enable-automation")
        options.add_argument("--no-sandbox")
        options.add_argument("--disable-dev-shm-usage")
        options.add_argument("--disable-browser-side-navigation")
        options.add_experimental_option("excludeSwitches", ["enable-automation"])
        options.add_experimental_option('useAutomationExtension', False)
        options.add_experimental_option('excludeSwitches', ['enable-logging'])
        
        page_count = 0  # Counter for pages scanned

        try:
            with webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options) as driver:
                driver.set_page_load_timeout(30)  # Set a longer timeout value (e.g., 30 seconds)

                # Read CSV file containing URLs
                url_df = pd.read_csv(csv_path)
                # Remove specific unwanted URL
                url_df = url_df.drop(url_df[url_df['url'] == "https://forums.hardwarezone.com.sg/forums/eat-drink-man-woman.16/"].index)

                # Iterate through each URL
                for index, row in tqdm(url_df.iterrows()):
                    # Filter by aspect term

                    if row.aspect != aspect_term:
                        continue

                    # Initialize lists to store data inside the loop
                    aspects_list = []
                    urls_list = []
                    titles_list = []
                    contents_list = []
                    authors_list = []
                    timeStamps_list = []
                    memberships_list = []

                    try:
                        # Visit the URL
                        driver.get(row.url)
                        
                        
                        try:
                            if page_count%10 == 0:
                                buttons = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.TAG_NAME, "button")))
                                
                                # Click on the first button found
                                if buttons:
                                    first_button = buttons[0]
                                    first_button.click()
                                else:
                                    print("No buttons found on the page.")
                                
                        except TimeoutException:
                            print("Timeout occurred while waiting for buttons to be located.")
                        except Exception as e:
                            print(f"An error occurred: {e}")



                        try:
                            # Find the first page link
                            thread_first_page = driver.find_element("xpath","//ul[@class='pageNav-main']/li/a")
                            driver.get(thread_first_page.get_attribute("href"))
                        except Exception as e:
                            #print(driver.find_element("xpath","//h1[@class='p-title-value']").text, "\t, this thread Only one pages")
                            pass

                        # Extract data from the page
                        contents_web = driver.find_elements("xpath","//div[@class='bbWrapper']")
                        for content in contents_web:
                            contents_list.append(content.text)
                            titles_list.append(driver.find_element("xpath","//h1[@class='p-title-value']").text)
                            urls_list.append(row.url)
                            aspects_list.append(row.aspect)

                        authors_web = driver.find_elements("xpath","//a[@class='username ']")
                        for author in authors_web:
                            authors_list.append(author.text)

                        timeStamps_web = driver.find_elements("xpath","//time[@class='u-dt']")
                        for timeStamp in timeStamps_web:
                            timeStamps_list.append(timeStamp.text)

                        memberships_web = driver.find_elements("xpath","//h5[@class='userTitle message-userTitle']")
                        for membership in memberships_web:
                            memberships_list.append(membership.text)

                        # Increment page count
                        page_count += 1

                        # Restart the browser after every 20 pages scanned
                        

                        # Iterating over pages until no next page is found or a limit is reached
                        flag = True
                        cnt = 0
                        while flag and cnt <= 100:
                            try:
                                forum_first_page = driver.find_element("xpath","//a[@class='pageNav-jump pageNav-jump--next']")
                                driver.get(forum_first_page.get_attribute("href"))
                                time.sleep(1)

                                contents_web = driver.find_elements("xpath","//div[@class='bbWrapper']")
                                for content in contents_web:
                                    contents_list.append(content.text)
                                    titles_list.append(driver.find_element("xpath","//h1[@class='p-title-value']").text)
                                    urls_list.append(row.url)
                                    aspects_list.append(row.aspect)

                                authors_web = driver.find_elements("xpath","//a[@class='username ']")
                                for author in authors_web:
                                    authors_list.append(author.text)

                                timeStamps_web = driver.find_elements("xpath","//time[@class='u-dt']")
                                for timeStamp in timeStamps_web:
                                    timeStamps_list.append(timeStamp.text)

                                memberships_web = driver.find_elements("xpath","//h5[@class='userTitle message-userTitle']")
                                for membership in memberships_web:
                                    memberships_list.append(membership.text)
                                cnt+=1
                            except Exception as e: 
                                flag = False
                                cnt = 0
                                print("Websites Name:\t", driver.find_element("xpath","//h1[@class='p-title-value']").text, ', We encounter a problem in this websites or we come the end the this thread')
                                time.sleep(1)
                        if page_count % 10 == 0:
                            #print('10 rows reached')
                            driver.close()
                            time.sleep(5)
                            driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)
                            driver.set_page_load_timeout(30)
                    except TimeoutException as te:
                        print("TimeoutException occurred:", str(te))

                    # Create a DataFrame and append to CSV
                    hardware_zone_df = pd.DataFrame(list(zip(urls_list, titles_list, contents_list,
                                                                authors_list, timeStamps_list, memberships_list, aspects_list)),
                                                        columns=['url', 'title', 'content', 'author', 
                                                                    'time_stamp', 'membership', "aspect"])
                    hardware_zone_df.to_csv("E:/Critical Inquiry/Code/Data Scraping/data/hwz_thread_all_data_" + aspect_term + '.csv', mode='a', index=False, header=False)
        except Exception as e:
            print("An error occurred:", str(e))

    
    def search_items(self, aspect_items: list):
        for item in aspect_items:
            self.getAndSave_url(item)
        
        self.combine_data()
    
    def combine_url_data(self):
        all_data = glob.glob(os.path.join("E:/Critical Inquiry/Code/Data Scraping/data/", "hwz_data_*.csv"))
        df_from_each_file = (pd.read_csv(f, sep=',') for f in all_data)
        df_merged = pd.concat(df_from_each_file, ignore_index=True)
        df_merged.drop_duplicates()
        c=0
        # Drop noise pages
        for index, row in tqdm(df_merged.iterrows()):
            cnt = 1
            if c == 20:
                self.driver.quit()
                options = webdriver.ChromeOptions() 
                options.add_argument('log-level=3')
                options.add_argument("start-maximized")
                options.add_argument("--disable-gpu")
                options.add_argument("enable-automation")
                options.add_argument("--headless")
                options.add_argument("--no-sandbox")
                options.add_argument("--disable-dev-shm-usage")
                options.add_argument("--disable-browser-side-navigation")
                options.add_experimental_option("excludeSwitches", ["enable-automation"])
                options.add_experimental_option('useAutomationExtension', False)
                options.add_experimental_option('excludeSwitches', ['enable-logging'])
                self.driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)
                c=0
            c+=1
            self.driver.get(row.url)
            try:
                pages = self.driver.find_elements("xpath","//li[@class='pageNav-page ']/a")
                for page in pages: 
                    cnt = max(int(page.text), cnt)
            except:
                cnt = 1
                
            if cnt >= 100:
                df_merged.drop(index, inplace=True)
            time.sleep(1)
        
        df_merged.to_csv("E:/Critical Inquiry/Code/Data Scraping/data/hwz_url_data.csv", index=False)
        

    def combine_data(self):
        all_data = glob.glob(os.path.join("E:/Critical Inquiry/Code/Data Scraping/data/", "hwz_thread_all_data_*.csv"))
        df_from_each_file = (pd.read_csv(f, header=None, sep=',') for f in all_data)  # Set header=None to read files without column names
        df_merged = pd.concat(df_from_each_file, ignore_index=True)
        df_merged.drop_duplicates(subset=[df_merged.columns[2]], inplace=True)  # Select the 3rd column by position
        df_merged.to_csv("E:/Critical Inquiry/Code/Data Scraping/data/hwz_all_data.csv", index=False)

In [3]:
#aspect_terms = ["lgbtq", "lgbt", "gay", "lesbian", "bisexual", "transgender", "queer", "pink dot", "pride month", "section 377A", "same-sex marriage"]
crawler = HardwarezoneCrawler()
#crawler.combine_url_data()
data_df = pd.read_csv("E:/Critical Inquiry/Code/Data Scraping/data/hwz_url_data.csv")
aspect_terms = dict(data_df["aspect"].value_counts()).keys()
print("All aspect terms:\t" + str(aspect_terms))
for aspect in aspect_terms:
    print("Aspect is :\t" + aspect)
    crawler.getReviews_content("E:/Critical Inquiry/Code/Data Scraping/data/hwz_url_data.csv", aspect)

All aspect terms:	dict_keys(['bisexual', 'gay', 'lgbt', 'pink dot', 'pride month', 'section 377A', 'transgender', 'lesbian', 'lgbtq', 'same-sex marriage', 'queer'])
Aspect is :	bisexual


0it [00:00, ?it/s]

Websites Name:	 Glgt bf broke back with gf father , We encounter a problem in this websites or we come the end the this thread


1it [00:12, 12.54s/it]

Websites Name:	 Section 377A review: Govt considering how to safeguard 'current legal position on marriage', says Shanmugam , We encounter a problem in this websites or we come the end the this thread


...

Websites Name:	 Who Cannot find gf becos of height? Do u find its dam unfair? , We encounter a problem in this websites or we come the end the this thread


1034it [1:20:30,  4.67s/it]


Aspect is :	queer


688it [05:21,  2.14it/s]


An error occurred: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=121.0.6167.161)
Stacktrace:
	GetHandleVerifier [0x00651673+52979]
	(No symbol) [0x005D7961]
	(No symbol) [0x004BDD3D]
	(No symbol) [0x004AE8A3]
	(No symbol) [0x004AE449]
	(No symbol) [0x004BF970]
	(No symbol) [0x0052ACD6]
	(No symbol) [0x00514286]
	(No symbol) [0x004EC063]
	(No symbol) [0x004ECECD]
	GetHandleVerifier [0x00968D83+3294723]
	GetHandleVerifier [0x009A6CC2+3548482]
	GetHandleVerifier [0x009A1C9C+3527964]
	GetHandleVerifier [0x006E870E+671630]
	(No symbol) [0x005E1EB4]
	(No symbol) [0x005DD808]
	(No symbol) [0x005DD92D]
	(No symbol) [0x005CF7E0]
	BaseThreadInitThunk [0x75EF7BA9+25]
	RtlInitializeExceptionChain [0x7722BD2B+107]
	RtlClearBits [0x7722BCAF+191]



In [4]:
crawler.getReviews_content("E:/Critical Inquiry/Code/Data Scraping/data/hwz_url_data.csv", "queer")

688it [00:49, 13.90it/s]


KeyboardInterrupt: 

In [19]:
crawler = HardwarezoneCrawler()
crawler.combine_data()

In [ ]:
file_path = 'E:/Critical Inquiry/Code/Data Scraping/data/hwz_all_data.csv'
column_names=['url', 'title', 'content', 'author', 'time_stamp', 'membership', "aspect"]
df = pd.read_csv(file_path, header=None)
    
# Add column names
df.columns = column_names

# Rewrite the file with column names included
df.to_csv(file_path, index=False)